### Get Cloud outage data into dataframe

First import the json and define column names and there mapping to json

In [1]:
import json

with open("../json-data/incidents.json") as file:
    incidents_json = json.load(file)

Create data frame and copy over the data

In [2]:
import pandas as pd
from datetime import datetime, timedelta

COLS = {"id": "id",    
        "start": "begin",
        "end": "end",
        "severity": "severity",
        "status": "status_impact",
        "location": ("currently_affected_locations", "previously_affected_locations"),
        "service_name": "service_name",
        "number_affected_products": "affected_products",
    }

df_incidents = pd.DataFrame(data=[], columns=COLS.keys())

def add1hour(starttime: str):
    converted = datetime.strptime(starttime, '%Y-%m-%dT%H:%M:%S+00:00')
    #print(converted)
    time_change = timedelta(minutes=60)
    new_time = converted + time_change
    #print(new_time)
    endtime = new_time.strftime('%Y-%m-%dT%H:%M:%S+00:00')
    #print(endtime)
    return endtime

for incident in incidents_json:
    locations = [x["id"] for x in incident.get(COLS["location"][0]) + incident.get(COLS["location"][1], None)]
    # print([item for item in locations if item not in data_centers["id"].to_list()])
    # countries = [data_centers.at[id, "country"] for id in locations]
    
    row = {
        "id": incident.get(COLS["id"], None),
        "start": incident.get(COLS["start"], None),
        "end": incident.get(COLS["end"], add1hour(incident.get(COLS["start"]))),
        "severity": incident.get(COLS["severity"], None),
        "location": locations,
        # "country": countries,
        "service_name": incident.get(COLS["service_name"]),
        "status": incident.get(COLS["status"], None),
        "number_affected_products": len(incident.get(COLS["number_affected_products"], []))
    }
    df_incidents.loc[len(df_incidents)] = row


Explode list of countries into multiple rows and add datacenter information

In [3]:
data_centers = pd.read_csv("../data/data-centers-rough.csv", dtype={
    'city': 'string','country':'string','id':'string'})
data_centers = data_centers.set_index("id")
data_centers.index.name = 'location'

df_exploded = df_incidents.explode("location")
df_incidentsWithLocation = pd.merge(df_exploded, data_centers, how="left", on="location")

Drop duplicate countries

In [4]:
print(df_incidentsWithLocation.shape)
df_incidentsWithCountry = df_incidentsWithLocation.drop_duplicates(subset=["id", "country"])
print(df_incidentsWithCountry.shape)

(3868, 10)
(2644, 10)


Create incidents table with incidents per day

In [5]:
# Expand DataFrame from range of dates to one day per row
df_incidentsWithCountry['from1'] = pd.to_datetime(df_incidentsWithCountry['start']).dt.date # dt.date looses hours and minutes information
df_incidentsWithCountry['to1'] = pd.to_datetime(df_incidentsWithCountry['end']).dt.date # dt.date looses hours and minutes information
L = [pd.Series(r.id, pd.date_range(r.from1, r.to1, freq='D')) for r in df_incidentsWithCountry.itertuples()]
df_idOnDateRange = pd.concat(L).reset_index()
df_idOnDateRange.columns = ['date', 'id']
# duplicates happended because of exploded country rows
df_idOnDateRange = df_idOnDateRange.drop_duplicates(["date", "id"])

df_incidentsPerDay = pd.merge(df_idOnDateRange, df_incidentsWithCountry, on="id", how="inner")
df_incidentsPerDay = df_incidentsPerDay.drop(columns=["from1", "to1"])

/var/folders/fc/nmrtwsyj4cx9_51gmbxm2lb80000gn/T/ipykernel_83097/2275863993.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_incidentsWithCountry['from1'] = pd.to_datetime(df_incidentsWithCountry['start']).dt.date # dt.date looses hours and minutes information
/var/folders/fc/nmrtwsyj4cx9_51gmbxm2lb80000gn/T/ipykernel_83097/2275863993.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_incidentsWithCountry['to1'] = pd.to_datetime(df_incidentsWithCountry['end']).dt.date # dt.date looses hours and mi

In [280]:
print(df_idOnDateRange.shape)
print(df_incidentsPerDay.shape)

(636, 2)
(5723, 11)


Create dataframe with row for each day x country

In [6]:
# Create df with only days
dates = pd.date_range(start='2022-09-15', end='2023-09-17', freq='D').to_frame(name='date')
# create df with only countries
countries = df_incidentsWithCountry["country"].drop_duplicates()
dates_countries = dates.merge(countries, how='cross')
print(dates_countries.shape)

df = pd.merge(dates_countries, df_incidentsPerDay, on=['date', 'country'], how='left')
print(df.shape)

(10304, 2)
(11726, 11)


In [282]:
print(df.head())

        date      country                    id                      start  \
0 2022-09-15       Taiwan  urNR4xD4gBNsyaZj3W1i  2022-09-15T23:01:57+00:00   
1 2022-09-15    Hong Kong  urNR4xD4gBNsyaZj3W1i  2022-09-15T23:01:57+00:00   
2 2022-09-15        Japan  urNR4xD4gBNsyaZj3W1i  2022-09-15T23:01:57+00:00   
3 2022-09-15  South Korea  urNR4xD4gBNsyaZj3W1i  2022-09-15T23:01:57+00:00   
4 2022-09-15        India  urNR4xD4gBNsyaZj3W1i  2022-09-15T23:01:57+00:00   

                         end severity               status         location  \
0  2022-09-29T20:49:04+00:00      low  SERVICE_INFORMATION       asia-east1   
1  2022-09-29T20:49:04+00:00      low  SERVICE_INFORMATION       asia-east2   
2  2022-09-29T20:49:04+00:00      low  SERVICE_INFORMATION  asia-northeast1   
3  2022-09-29T20:49:04+00:00      low  SERVICE_INFORMATION  asia-northeast3   
4  2022-09-29T20:49:04+00:00      low  SERVICE_INFORMATION      asia-south1   

               service_name  number_affected_products   

Export as csv table

In [7]:
df.to_csv("../data/incidentsByDateCountry.csv")